# Pandas  分组

任何分组操作都涉及原始对象的以下操作之一:

* 分割对象
* 应用一个函数
* 结合的结果

在许多情况下，我们将数据分成多个集合，并在每个子集上应用一些函数:

* 聚合 - 计算汇总统计
* 转换 - 执行一些特定于组的操作
* 过滤 - 在某些情况下丢弃数据

In [2]:
import pandas as pd

ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
                     'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
            'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
            'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
            'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)
print(df)

    Points  Rank    Team  Year
0      876     1  Riders  2014
1      789     2  Riders  2015
2      863     2  Devils  2014
3      673     3  Devils  2015
4      741     3   Kings  2014
5      812     4   kings  2015
6      756     1   Kings  2016
7      788     1   Kings  2017
8      694     2  Riders  2016
9      701     4  Royals  2014
10     804     1  Royals  2015
11     690     2  Riders  2017


## 将数据拆分成组

Pandas 对象可以分成任何对象，有多种方式来拆分对象:

* obj.groupby( 'key' )
* obj.groupby([ 'key1', 'key2' ])
* obj.groupby(key, axis=1)

In [3]:
print(df.groupby('Team'))

In [4]:
# 查看分组
df.groupby('Team').groups

{'Devils': Int64Index([2, 3], dtype='int64'),
 'Kings': Int64Index([4, 6, 7], dtype='int64'),
 'Riders': Int64Index([0, 1, 8, 11], dtype='int64'),
 'Royals': Int64Index([9, 10], dtype='int64'),
 'kings': Int64Index([5], dtype='int64')}

In [5]:
# 按多列分组
df.groupby(['Team', 'Year']).groups

{('Devils', 2014): Int64Index([2], dtype='int64'),
 ('Devils', 2015): Int64Index([3], dtype='int64'),
 ('Kings', 2014): Int64Index([4], dtype='int64'),
 ('Kings', 2016): Int64Index([6], dtype='int64'),
 ('Kings', 2017): Int64Index([7], dtype='int64'),
 ('Riders', 2014): Int64Index([0], dtype='int64'),
 ('Riders', 2015): Int64Index([1], dtype='int64'),
 ('Riders', 2016): Int64Index([8], dtype='int64'),
 ('Riders', 2017): Int64Index([11], dtype='int64'),
 ('Royals', 2014): Int64Index([9], dtype='int64'),
 ('Royals', 2015): Int64Index([10], dtype='int64'),
 ('kings', 2015): Int64Index([5], dtype='int64')}

## 迭代遍历分组

使用 groupby 对象，可以遍历类似 itertools.obj 的对象

In [6]:
grouped = df.groupby('Year')
for name, group in grouped:
    print(name, ":\n", group)

2014 :
    Points  Rank    Team  Year
0     876     1  Riders  2014
2     863     2  Devils  2014
4     741     3   Kings  2014
9     701     4  Royals  2014
2015 :
     Points  Rank    Team  Year
1      789     2  Riders  2015
3      673     3  Devils  2015
5      812     4   kings  2015
10     804     1  Royals  2015
2016 :
    Points  Rank    Team  Year
6     756     1   Kings  2016
8     694     2  Riders  2016
2017 :
     Points  Rank    Team  Year
7      788     1   Kings  2017
11     690     2  Riders  2017


## 选择一个分组

使用 get_group() 方法，可以选择一个组

In [7]:
print(grouped.get_group(2017))
print(type(grouped.get_group(2017)))

    Points  Rank    Team  Year
7      788     1   Kings  2017
11     690     2  Riders  2017
<class 'pandas.core.frame.DataFrame'>


## 聚合

聚合函数为每个组返回单个聚合值，当创建了分组对象，就可以对分组数据执行多个聚合操作。一个比较常用的是通过聚合或等效的 agg 方法聚合

In [14]:
import numpy as np

grouped = df.groupby('Year')
print(grouped['Points'].agg(np.mean))
print(grouped['Points'].agg(np.sum))

Year
2014    795.25
2015    769.50
2016    725.00
2017    739.00
Name: Points, dtype: float64
Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64


### 另外一种查看每个分组的大小的方法是应用  size() 函数

In [9]:
grouped = df.groupby('Team')
print(grouped.agg(np.size))

        Points  Rank  Year
Team                      
Devils       2     2     2
Kings        3     3     3
Riders       4     4     4
Royals       2     2     2
kings        1     1     1


### 一次应用多个聚合函数

通过分组序列，还可以传递函数的列表或字典进行聚合，并生成 DataFrame 作为输出

In [10]:
grouped = df.groupby('Team')
agg = grouped['Points'].agg([np.sum, np.mean, np.std])
print(agg)

         sum        mean         std
Team                                
Devils  1536  768.000000  134.350288
Kings   2285  761.666667   24.006943
Riders  3049  762.250000   88.567771
Royals  1505  752.500000   72.831998
kings    812  812.000000         NaN


## 转换

分组或列上的转换返回索引大小与被分组的索引相同的对象，因此，转换应该返回与组块大小相同的结果。

In [11]:
grouped = df.groupby('Team')
score = lambda x: (x - x.mean()) / x.std() * 10
print(grouped.transform(score))

       Points       Rank       Year
0   12.843272 -15.000000 -11.618950
1    3.020286   5.000000  -3.872983
2    7.071068  -7.071068  -7.071068
3   -7.071068   7.071068   7.071068
4   -8.608621  11.547005 -10.910895
5         NaN        NaN        NaN
6   -2.360428  -5.773503   2.182179
7   10.969049  -5.773503   8.728716
8   -7.705963   5.000000   3.872983
9   -7.071068   7.071068  -7.071068
10   7.071068  -7.071068   7.071068
11  -8.157595   5.000000  11.618950


## 过滤

过滤根据定义的标准过滤数据并返回数据的子集。 filter() 函数用于过滤数据。

In [12]:
df_filter = df.groupby('Team').filter(lambda x: len(x) >= 3)
print(df_filter)

    Points  Rank    Team  Year
0      876     1  Riders  2014
1      789     2  Riders  2015
4      741     3   Kings  2014
6      756     1   Kings  2016
7      788     1   Kings  2017
8      694     2  Riders  2016
11     690     2  Riders  2017
